In [ ]:
import os
import csv
from IPython.display import display, HTML
import subprocess

def check_parameters(ion_dist_cutoff, mlengine, partner_1, partner_2):
    """
    检查参数的有效性
    """
    if ion_dist_cutoff is None:
        display(HTML(f"<span style='color: red; font-size: 14px; font-weight: bold;'>Warning: There is no ion_dist_cutoff. Please, input a value! </span>"))
        return False
    elif ion_dist_cutoff < 0:
        display(HTML(f"<span style='color: red; font-size: 14px; font-weight: bold;'>Warning: Unavailable ion_dist_cutoff. Please, input a positive value!</span>"))
        return False
    elif mlengine not in ["sl", "lr", "en", "sv", "dt", "kn", "ad", "bg", "rf", "et", "xb"]:
        display(HTML(f"<span style='color: red; font-size: 14px; font-weight: bold;'>Warning: Unavailable mlengine. Please, input the models described in the details!</span>"))
        return False
    elif mlengine == "":
        display(HTML(f"<span style='color: red; font-size: 14px; font-weight: bold;'>Warning: There is no mlegine. Please, input an avaiable one! </span>"))
        return False
    elif partner_1 == "" or partner_2 == "":
        display(HTML(f"<span style='color: red; font-size: 14px; font-weight: bold;'>Warning: Empty partners. Please, fill in them correctly!</span>"))
        return False
    return True

def generate_command(frcmod_struct, frcmod_scores, ion_dist_cutoff, mlengine, partner_1, partner_2, destination_path):
    """
    根据参数生成命令
    """
    base_command = f'python3 /public/home/design2/soft/PBEE/pbee.py --ipdb {os.path.join(destination_path, "*.pdb")} --partner1 {partner_1} --partner2 {partner_2} --odir {os.path.join(destination_path)}'
    if frcmod_struct:
        base_command += " --frcmod_struct"
    if frcmod_scores:
        base_command += " --frcmod_scores"
    if ion_dist_cutoff != 2:
        base_command += f" --ion_dist_cutoff {ion_dist_cutoff}"
    if mlengine != "sl":
        base_command += f" --mlengine {mlengine}"
    return base_command

def run_command_and_capture_output(command):
    """
    运行命令并捕获输出
    """
    try:
        result = subprocess.run(command, shell=True, capture_output=True, text=True)
        return result.stdout.strip()
    except Exception as e:
        print(f"Error running command: {e}")
        return None

# 代码输入目标路径
destination_path = "/public/home/design2/project/rf/05_ddG_calculation/nano"
os.makedirs(destination_path, exist_ok=True)

# 直接在代码中指定 partner_1 和 partner_2 的值
partner_1 = "A"
partner_2 = "B"

# 直接在代码中指定 frcmod_struct 和 frcmod_scores 的值
frcmod_struct = False
frcmod_scores = False

# 直接在代码中指定离子距离截断值
ion_dist_cutoff = 8

# 直接在代码中指定机器学习引擎
mlengine = "sl"

# 定义 CSV 文件路径
csv_file_path = os.path.join(destination_path, "output.csv")

# 打开 CSV 文件以写入数据
with open(csv_file_path, 'w', newline='') as csvfile:
    fieldnames = ['Command', 'Output']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    # 写入 CSV 文件的表头
    writer.writeheader()
    print(f"写入表头: {fieldnames}")

    # 遍历四种情况
    for fs, fs_scores in [(False, False), (True, False), (False, True), (True, True)]:
        frcmod_struct = fs
        frcmod_scores = fs_scores

        if not check_parameters(ion_dist_cutoff, mlengine, partner_1, partner_2):
            continue

        command = generate_command(frcmod_struct, frcmod_scores, ion_dist_cutoff, mlengine, partner_1, partner_2, destination_path)
        output = run_command_and_capture_output(command)
        writer.writerow({'Command': command, 'Output': output})
        print(f"写入行: Command={command}, Output={output}")
